In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", palette="coolwarm")
import numpy as np

In [18]:
area_sale = pd.read_csv('area_cat_cust.csv', sep=';')
area_mapping = pd.read_csv('mapping.csv', sep=';')

In [2]:
# Dataset gerado após análise dos dados sales
basket = pd.read_csv('basket.csv', sep=';')

# Metrics

In [19]:
area_sale.head()

,area,category_name,external_id
0,311,1,7430
1,314,1,3099
2,315,3,6883
3,317,2,4718
4,322,1,6082


In [21]:
area_mapping.head()

,area,category_id,category_name,duration,quantity
0,X,3270.0,CONFECCAO,14865.0,1496.0
1,311,3997.0,BEBIDAS ALCOOLICAS,409731.0,34144.0
2,X,6456.0,CONGELADO,41309.0,8703.0
3,X,1318.0,CARNES,41309.0,8703.0
4,X,1373.0,TABACARIA,NaN,NaN


In [22]:
mapping_grouped = area_mapping.groupby('area')['duration','quantity'].sum().reset_index()

In [24]:
mapping_grouped.head()

,area,duration,quantity
0,311,409731.0,34144.0
1,314,314915.0,18312.0
2,315,297765.0,22077.0
3,317,275514.0,16942.0
4,322,881205.0,33861.0


In [25]:
summary_area = pd.merge(area_sale,
                 mapping_grouped,
                 on='area', 
                 how='left')

In [26]:
summary_area.head()

,area,category_name,external_id,duration,quantity
0,311,1,7430,409731.0,34144.0
1,314,1,3099,314915.0,18312.0
2,315,3,6883,297765.0,22077.0
3,317,2,4718,275514.0,16942.0
4,322,1,6082,881205.0,33861.0


In [27]:
summary_area.isnull().sum()

area             0
category_name    0
external_id      0
duration         0
quantity         0
dtype: int64

Algumas não não tiveram compras registradas ou a categoria dos produtos não estava mapeada no arquivo mapping, para estes casos estou optando por exclui-los, e deixar somente os registros que poderei calcular as métricas.

In [28]:
summary_area.dropna(inplace=True)

In [29]:
summary_area['conversion'] = (summary_area['external_id'] / summary_area['quantity'])*100

In [30]:
summary_area

,area,category_name,external_id,duration,quantity,conversion
0,311,1,7430,409731.0,34144.0,21.760778
1,314,1,3099,314915.0,18312.0,16.923329
2,315,3,6883,297765.0,22077.0,31.177243
3,317,2,4718,275514.0,16942.0,27.847952
4,322,1,6082,881205.0,33861.0,17.961667
5,324,3,6414,1258674.0,57054.0,11.241981
6,329,3,10176,124608.0,7881.0,129.120670
7,330,3,9384,52105.0,3662.0,256.253413
8,332,1,8554,922711.0,22475.0,38.060067
9,333,5,1646,211125.0,13890.0,11.850252


In [31]:
summary_area.rename(index=str, columns={"external_id": "basket", 
                                        "category_name": "total_category",
                                        "quantity": "visitor"}, inplace=True)

#### 1. Exposure : How many people were exposed to that category or area? Meaning, how many people were identified on the area of that item?

In [32]:
# 10 áreas com mais pessoas expostas
summary_area[['area','visitor']].sort_values('visitor', ascending=False).head(10)

,area,visitor
27,364,470534.0
36,400,135053.0
11,341,133808.0
32,389,115595.0
28,365,111432.0
25,361,108533.0
12,344,86442.0
26,362,85304.0
38,421,85246.0
16,350,81175.0


In [56]:
# Média geral de visitantes 
np.round(np.mean(summary_area.visitor))

53119.0

#### 2. Engagement : What is the average engagement power of a given category or area? Meaning, how long does someone, on average, spend on a given area? It's defined as the average time spent by shoppers spent on each area.

In [40]:
summary_area['exposure'] = np.round(summary_area['duration'] / summary_area['visitor'], decimals=1)
summary_area.sort_values('exposure', ascending=False)

,area,total_category,basket,duration,visitor,conversion,exposure
8,332,1,8554,922711.0,22475.0,38.060067,41.1
20,356,1,7487,60064.0,2192.0,341.560219,27.4
4,322,1,6082,881205.0,33861.0,17.961667,26.0
15,348,1,8580,1011057.0,42185.0,20.338983,24.0
5,324,3,6414,1258674.0,57054.0,11.241981,22.1
12,344,3,4231,1832217.0,86442.0,4.894611,21.2
14,346,2,5345,931817.0,48480.0,11.025165,19.2
26,362,6,2871,1615386.0,85304.0,3.365610,18.9
17,352,3,5642,1145749.0,62627.0,9.008894,18.3
16,350,8,5511,1465461.0,81175.0,6.789036,18.1


In [57]:
# Média geral de tempo de exposição
np.round(np.mean(summary_area.exposure))

16.0

In [44]:
fig = plt.figure(figsize=(18,6), dpi=1600) 
sns.set_color_codes("dark")
# Cria o subplot1 
ax1 = plt.subplot(2,1,1)
fig=sns.barplot(summary_area['area'], summary_area['visitor'], color="b");

# Define o título do gráfico
plt.title("Total de pessoas expostas à area")

# Cria o subplot2 
ax2 = plt.subplot(2,1,2)

sns.barplot(summary_area['area'], summary_area['basket'], color="g");
plt.title("Total de compras por área");


#### 3. Conversion AREA : On average, how many people need to be exposed to an item so one purchase will be made? This can be defined as #_baskets_sold/exposure (We define one basket as one shopper that purchased that item)

In [50]:
summary_area[['area','conversion']].sort_values('conversion', ascending=False).head(15)

,area,conversion
20,356,341.560219
7,330,256.253413
39,422,134.005472
6,329,129.120670
19,355,64.067398
8,332,38.060067
2,315,31.177243
40,425,29.367002
3,317,27.847952
24,360,25.585697


In [58]:
# Média geral do ratio
np.round(np.mean(summary_area.conversion))

33.0

As áreas 329, 330, 356 e 422  estão incosistentes, porque elas tiveram um total de compras superior ao número de visitantes. Pode ter havido problemas nos registros gravados neste dia? Parte disso é consequência dos outliers identificados, mas não tratados.

In [51]:
summary_area.loc[summary_area['conversion']>95]

,area,total_category,basket,duration,visitor,conversion,exposure
6,329,3,10176,124608.0,7881.0,129.120670,15.8
7,330,3,9384,52105.0,3662.0,256.253413,14.2
20,356,1,7487,60064.0,2192.0,341.560219,27.4
39,422,2,13225,118782.0,9869.0,134.005472,12.0


In [52]:
area_mapping[area_mapping['area']=='356']

,area,category_id,category_name,duration,quantity
52,356,6727.0,IOGURTES,60064.0,2192.0


#### 3.1 Conversion CATEGORY : On average, how many people need to be exposed to an item so one purchase will be made? This can be defined as #_baskets_sold/exposure (We define one basket as one shopper that purchased that item)quantity

In [111]:
summary_category =area_mapping.groupby('category_name')['duration','quantity'].sum().reset_index().sort_values('quantity', ascending=False)

In [118]:
summary_category.dropna(inplace=True)

In [119]:
# 10 categorias com mais exposição
summary_category['exposure'] = np.round(summary_category['duration'] / summary_category['quantity'], decimals=1)
summary_category.sort_values('exposure', ascending=False).head(10)

,category_name,duration,quantity,exposure
37,FRUTAS,922711.0,22475.0,41.1
82,PRODUTOS SAZONAIS,811047.0,23941.0,33.9
43,IOGURTES,60064.0,2192.0,27.4
81,PRODUTOS PARA ROUPA,881205.0,33861.0,26.0
79,PRODUTOS PARA CASA,1011057.0,42185.0,24.0
86,SABONETES,419558.0,19018.0,22.1
40,HIGIENE ORAL,419558.0,19018.0,22.1
95,TALCO,419558.0,19018.0,22.1
58,MOLHO TOMATE E SIMILARES,610739.0,28814.0,21.2
103,VEGETAIS EM CONSERVA,610739.0,28814.0,21.2


In [120]:
# 10 categorias com menor exposição
summary_category.sort_values('exposure', ascending=False).tail(10)

,category_name,duration,quantity,exposure
102,VASOS,96085.0,9780.0,9.8
6,BEBIDAS NAO GASEIFICADAS,170198.0,19028.0,8.9
46,LEITES,92395.0,10506.0,8.8
66,PET SHOP,55295.0,6787.0,8.1
101,VASILHAME,53860.0,7002.0,7.7
98,UTENSILIOS PARA COZINHA,99679.0,13785.0,7.2
5,BEBIDAS GASEIFICADAS,4364.0,814.0,5.4
15,CARNES,41309.0,8703.0,4.7
22,CONGELADO,41309.0,8703.0,4.7
84,RESFRIADO,41309.0,8703.0,4.7


### Other metrics

In [87]:
#Total de visitantes no perído:
print('Total de visitantes: {0:,}'.format(summary_area.visitor.sum()))

Total de visitantes: 2,230,979.0


In [88]:
#Total de visitantes no perído:
print('Total de baskets: {0:,}'.format(# Dataset gerado após análise dos dados sales
basket = pd.read_csv('basket.csv', sep=';')))

Total de baskets: 227,274


In [89]:
#Total de visitantes no perído:
print('Total de areas: {0:,}'.format(summary_area.area.count()))

Total de areas: 42


In [101]:
#Units per transaction (UPT)= (Total de itens vendidos / total baskets)
print('Media de itens vendidos por transação: {0:.0f}'.format((basket.quantity.sum()/summary_area.visitor.sum())*100))

Media de itens vendidos por transação: 30 %


In [106]:
#(Size of an average shopping cart)
print('Media de itens vendidos por basket: {0:.0f}'.format(basket.quantity.mean()))

Media de itens vendidos por basket: 3


# Market Basket Analysis

OBS: Considerado na ánálise somente os produtos comprados nas áreas de interesse.

Métricas:
- support = probability 2 products
- confidence = support/probability item 1
- lift = confidence/probability item 2 - Rule importance 

In [9]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [17]:
basket_analysis = pd.read_csv('basket_analysis.csv', sep=';')

In [27]:
basket_analysis = basket_analysis.set_index('external_id')

In [39]:
# Extract frequent itemsets
frequent_items = apriori(basket_analysis, min_support=0.07, use_colnames=True)

In [40]:
# Find frequent patterns
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

In [44]:
rules.sort_values('support', ascending=False).head(10)

,antecedants,consequents,support,confidence,lift
48,(RESFRIADOS),(OLEO),0.535597,0.134236,1.062787
52,(RESFRIADOS),(OVOS),0.535597,0.145654,1.170193
64,(RESFRIADOS),(QUEIJOS),0.535597,0.137027,1.698771
24,(RESFRIADOS),(INFANTIL),0.535597,0.131656,1.567055
62,(RESFRIADOS),(PREPARO BOLOS-SORV-TORTAS),0.535597,0.132290,1.376759
40,(RESFRIADOS),(MATINAIS),0.535597,0.136689,1.057181
60,(RESFRIADOS),(PAPEL HIGIENICO),0.535597,0.140833,1.198874
20,(RESFRIADOS),(GRAOS E FARINACEOS),0.535597,0.137534,1.057793
44,(RESFRIADOS),(MOLHOS FRIOS),0.535597,0.138253,1.114789
16,(RESFRIADOS),(DESODORANTES E COLONIAS),0.535597,0.134532,1.670652


In [42]:
rules.sort_values('confidence', ascending=False).head(10)

,antecedants,consequents,support,confidence,lift
58,(QUEIJOS),(PAES E BOLOS),0.080662,0.927548,2.016968
65,(QUEIJOS),(RESFRIADOS),0.080662,0.909857,1.698771
17,(DESODORANTES E COLONIAS),(RESFRIADOS),0.080526,0.894796,1.670652
14,(DESODORANTES E COLONIAS),(PAES E BOLOS),0.080526,0.884669,1.923727
25,(INFANTIL),(RESFRIADOS),0.084015,0.839310,1.567055
63,(PREPARO BOLOS-SORV-TORTAS),(RESFRIADOS),0.096088,0.737388,1.376759
56,(PREPARO BOLOS-SORV-TORTAS),(PAES E BOLOS),0.096088,0.728666,1.584494
61,(PAPEL HIGIENICO),(RESFRIADOS),0.117471,0.642113,1.198874
54,(PAPEL HIGIENICO),(PAES E BOLOS),0.117471,0.638257,1.387899
50,(OVOS),(PAES E BOLOS),0.124471,0.630755,1.371587


In [43]:
rules.sort_values('lift', ascending=False).head(10)

,antecedants,consequents,support,confidence,lift
58,(QUEIJOS),(PAES E BOLOS),0.080662,0.927548,2.016968
59,(PAES E BOLOS),(QUEIJOS),0.459873,0.162693,2.016968
14,(DESODORANTES E COLONIAS),(PAES E BOLOS),0.080526,0.884669,1.923727
15,(PAES E BOLOS),(DESODORANTES E COLONIAS),0.459873,0.154911,1.923727
35,(LEGUMES),(PAPEL HIGIENICO),0.352799,0.204880,1.744084
34,(PAPEL HIGIENICO),(LEGUMES),0.117471,0.615310,1.744084
65,(QUEIJOS),(RESFRIADOS),0.080662,0.909857,1.698771
64,(RESFRIADOS),(QUEIJOS),0.535597,0.137027,1.698771
16,(RESFRIADOS),(DESODORANTES E COLONIAS),0.535597,0.134532,1.670652
17,(DESODORANTES E COLONIAS),(RESFRIADOS),0.080526,0.894796,1.670652


In [46]:
rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.80) ]

,antecedants,consequents,support,confidence,lift
14,(DESODORANTES E COLONIAS),(PAES E BOLOS),0.080526,0.884669,1.923727
17,(DESODORANTES E COLONIAS),(RESFRIADOS),0.080526,0.894796,1.670652
25,(INFANTIL),(RESFRIADOS),0.084015,0.839310,1.567055
58,(QUEIJOS),(PAES E BOLOS),0.080662,0.927548,2.016968
65,(QUEIJOS),(RESFRIADOS),0.080662,0.909857,1.698771


Foi estabelecido como medida de importância das regras um nível de confiança acima de 80% e um lift maior que 1.
Observa-se ótimas métricas para o relacionamento de:
1. QUEIJOS com PAES E BOLOS/RESFRIADOS
2. DESODORANTES E COLONIAS com PAES E BOLOS/RESFRIADOS
3. INFANTIL com RESFRIADOS

Dos relacionamentos citados acima, a categoria 'DESODORANTES E COLONIAS' estar associada a PAES E BOLOS/RESFRIADOS é a mais surpreendente.

Uma opção interessante para separar a amostra para este teste, seria separar os baskets por total de itens. Assim, poderiamos separar comprar rápidas(por exemplo até 10 itens) de compras mensais do cliente, onde é bem provável que tenha muitos itens relacionados sem muito significado.